
# Gesture Recognition Project

This project involves recognizing hand gestures performed in front of a smart TV using video data captured by a webcam. 
The goal is to create a deep learning model that classifies hand gestures into five categories, which will correspond to specific TV commands such as increasing volume, pausing, etc.

### Problem Statement
We need to build a model that processes videos of hand gestures and correctly classifies them into one of five categories using Conv3D or CNN-RNN architectures.


In [8]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os
from PIL import Image
from keras.utils import to_categorical

# Load the training and validation CSV files
train_csv = 'train.csv'
val_csv = 'val.csv'

train_df = pd.read_csv(train_csv)
val_df = pd.read_csv(val_csv)

# Clean up the data (split columns)
train_df_cleaned = train_df[train_df.columns[0]].str.split(';', expand=True)
val_df_cleaned = val_df[val_df.columns[0]].str.split(';', expand=True)
train_df_cleaned.columns = ['folder_name', 'gesture', 'label']
val_df_cleaned.columns = ['folder_name', 'gesture', 'label']
train_df_cleaned['label'] = train_df_cleaned['label'].astype(int)
val_df_cleaned['label'] = val_df_cleaned['label'].astype(int)

# Display cleaned data samples
print(train_df_cleaned.head())
print(val_df_cleaned.head())

# Define paths for training and validation data
train_path = 'train'
val_path = 'val'


                                folder_name         gesture  label
0  WIN_20180925_17_18_28_Pro_Left_Swipe_new  Left_Swipe_new      0
1  WIN_20180925_17_18_56_Pro_Left_Swipe_new  Left_Swipe_new      0
2  WIN_20180925_17_19_51_Pro_Left_Swipe_new  Left_Swipe_new      0
3  WIN_20180925_17_20_14_Pro_Left_Swipe_new  Left_Swipe_new      0
4  WIN_20180925_17_21_28_Pro_Left_Swipe_new  Left_Swipe_new      0
                                folder_name         gesture  label
0  WIN_20180925_17_43_01_Pro_Left_Swipe_new  Left_Swipe_new      0
1  WIN_20180925_18_01_40_Pro_Left_Swipe_new  Left_Swipe_new      0
2  WIN_20180925_18_03_21_Pro_Left_Swipe_new  Left_Swipe_new      0
3  WIN_20180926_16_46_22_Pro_Left_Swipe_new  Left_Swipe_new      0
4  WIN_20180926_16_47_09_Pro_Left_Swipe_new  Left_Swipe_new      0


In [9]:
from PIL import Image

def generator(source_path, folder_list, batch_size, image_size=(64, 64), frames=15, crop_size=(100, 100)):
    """
    Generator function to yield batches of video frames and corresponding labels.
    Args:
        source_path (str): Path to source directory containing video folders.
        folder_list (list): List of folders (videos) for each batch.
        batch_size (int): Number of videos to process in a batch.
        image_size (tuple): Desired (height, width) to resize the images.
        frames (int): Number of frames to sample from each video.
        crop_size (tuple): Desired (height, width) after cropping.
    """
    print('Source path =', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size
        
        for batch in range(num_batches):
            batch_data = np.zeros((batch_size, frames, image_size[0], image_size[1], 3))
            batch_labels = np.zeros((batch_size, 5))  # One-hot encoding of labels
            
            for folder in range(batch_size):
                video_folder = source_path + '/' + t[folder + (batch * batch_size)][0]
                imgs = sorted(os.listdir(video_folder))  # Sorted to maintain sequence order
                
                # Step 1: Sample frames - get evenly spaced frames to capture entire gesture sequence
                selected_frames = np.linspace(0, len(imgs) - 1, frames, dtype=int)
                
                for idx, frame_idx in enumerate(selected_frames):
                    img_path = os.path.join(video_folder, imgs[frame_idx])
                    image = Image.open(img_path)
                    
                    # Step 2: Center crop image
                    width, height = image.size
                    left = (width - crop_size[0]) / 2
                    top = (height - crop_size[1]) / 2
                    right = (width + crop_size[0]) / 2
                    bottom = (height + crop_size[1]) / 2
                    image = image.crop((left, top, right, bottom))
                    
                    # Step 3: Resize and normalize
                    image = image.resize(image_size)
                    batch_data[folder, idx, :, :, :] = np.array(image) / 255.0  # Normalize pixel values

                # Get label and encode in one-hot format
                label = int(t[folder + (batch * batch_size)][2])
                batch_labels[folder, label] = 1

            yield batch_data, batch_labels
        
        # Handle remaining data points after full batches
        remaining_data_points = len(folder_list) % batch_size
        if remaining_data_points > 0:
            batch_data = np.zeros((remaining_data_points, frames, image_size[0], image_size[1], 3))
            batch_labels = np.zeros((remaining_data_points, 5))

            for folder in range(remaining_data_points):
                video_folder = source_path + '/' + t[folder + (num_batches * batch_size)][0]
                imgs = sorted(os.listdir(video_folder))
                selected_frames = np.linspace(0, len(imgs) - 1, frames, dtype=int)

                for idx, frame_idx in enumerate(selected_frames):
                    img_path = os.path.join(video_folder, imgs[frame_idx])
                    image = Image.open(img_path)

                    # Center crop and resize
                    width, height = image.size
                    left = (width - crop_size[0]) / 2
                    top = (height - crop_size[1]) / 2
                    right = (width + crop_size[0]) / 2
                    bottom = (height + crop_size[1]) / 2
                    image = image.crop((left, top, right, bottom))
                    image = image.resize(image_size)
                    batch_data[folder, idx, :, :, :] = np.array(image) / 255.0

                label = int(t[folder + (num_batches * batch_size)][2])
                batch_labels[folder, label] = 1

            yield batch_data, batch_labels




### Model Architecture

In this section, we will build two architectures for processing video sequences:

1. **Conv3D Model**: A 3D convolutional neural network for spatiotemporal feature extraction.
2. **CNN + RNN Model**: A 2D CNN combined with a GRU (Gated Recurrent Unit) to handle temporal sequences.


In [15]:
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization, Activation

def create_lightweight_conv3d_model(input_shape=(15, 64, 64, 3)):
    model = Sequential()
    
    # Reduced number of filters and kernel sizes
    model.add(Conv3D(16, kernel_size=(3, 3, 3), input_shape=input_shape, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    model.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    # Flatten layer
    model.add(Flatten())
    
    # Dynamically set the units for Dense layer based on output of Flatten layer
    flattened_output_shape = model.output_shape[1]  # Dynamically get flattened shape
    model.add(Dense(flattened_output_shape, activation='relu'))  # Adjust this number dynamically
    
    # Output layer for 5 classes
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))
    
    return model

# Instantiate and compile the lightweight Conv3D model
conv3d_model = create_lightweight_conv3d_model()
conv3d_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
conv3d_model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_15 (Conv3D)              │ (None, 15, 64, 64, 16) │         1,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 15, 64, 64, 16) │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_15 (Activation)      │ (None, 15, 64, 64, 16) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_15 (MaxPooling3D) │ (None, 7, 32, 32, 16)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_16 (Conv3D)              │ (None, 7, 32, 32, 32)  │        13,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 7, 32, 32, 32)  │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_16 (Activation)      │ (None, 7, 32, 32, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_16 (MaxPooling3D) │ (None, 3, 16, 16, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_17 (Conv3D)              │ (None, 3, 16, 16, 64)  │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 3, 16, 16, 64)  │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_17 (Activation)      │ (None, 3, 16, 16, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_17 (MaxPooling3D) │ (None, 1, 8, 8, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 5)              │        20,485 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,872,773 (64.36 MB)

 Trainable params: 16,872,549 (64.36 MB)

 Non-trainable params: 224 (896.00 B)

In [17]:

# CNN + RNN Model (GRU)
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, TimeDistributed, GRU, Dense, Flatten, Dropout, BatchNormalization, Activation

def create_cnn_rnn_model(input_shape=(30, 64, 64, 3)):
    model = Sequential()
    
    # TimeDistributed Conv2D layers for each frame
    model.add(TimeDistributed(Conv2D(32, kernel_size=(3, 3), padding='same'), input_shape=input_shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation('relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    
    model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), padding='same')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation('relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    
    # GRU layer for temporal sequence processing
    model.add(TimeDistributed(Flatten()))
    model.add(GRU(128, return_sequences=False))
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))  # 5 gesture classes
    
    return model

# Instantiate the model
cnn_rnn_model = create_cnn_rnn_model()
cnn_rnn_model.summary()


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_9              │ (None, 30, 64, 64, 32) │           896 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_10             │ (None, 30, 64, 64, 32) │           128 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_11             │ (None, 30, 64, 64, 32) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_12             │ (None, 30, 32, 32, 32) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_13             │ (None, 30, 32, 32, 64) │        18,496 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_14             │ (None, 30, 32, 32, 64) │           256 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_15             │ (None, 30, 32, 32, 64) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_16             │ (None, 30, 16, 16, 64) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_17             │ (None, 30, 16384)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 128)            │     6,341,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 512)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,429,765 (24.53 MB)

 Trainable params: 6,429,573 (24.53 MB)

 Non-trainable params: 192 (768.00 B)

In [18]:

# Compile and train the models
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Compile Conv3D model
conv3d_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

## checkpoint = ModelCheckpoint('conv3d_best_model.h5', monitor='val_loss', verbose=1, save_best_only=True)
# checkpoint = ModelCheckpoint('conv3d_best_model.keras', monitor='val_loss', verbose=1, save_best_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

# callbacks_list = [checkpoint, reduce_lr]

# Callbacks to save the best model and reduce learning rate if the model stops improving
checkpoint_model = ModelCheckpoint('conv3d_best_model.keras', monitor='val_loss', verbose=1, save_best_only=True)
checkpoint_weights = ModelCheckpoint('conv3d_best_weights.weights.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)
callbacks_list = [checkpoint_model, checkpoint_weights, reduce_lr]

# Set batch size and steps per epoch
batch_size = 32
steps_per_epoch = len(train_df_cleaned) // batch_size
validation_steps = len(val_df_cleaned) // batch_size

# Define generators for training and validation
train_generator = generator(train_path, train_df_cleaned.values, batch_size=batch_size)
val_generator = generator(val_path, val_df_cleaned.values, batch_size=batch_size)

# Train the Conv3D model
conv3d_model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, 
                 verbose=1, callbacks=callbacks_list, validation_data=val_generator, 
                 validation_steps=validation_steps)

Source path = train ; batch size = 32
Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - categorical_accuracy: 0.2403 - loss: 18.5076Source path = val ; batch size = 32

Epoch 1: val_loss improved from inf to 1.59916, saving model to conv3d_best_model.keras

Epoch 1: val_loss improved from inf to 1.59916, saving model to conv3d_best_weights.weights.h5
20/20 ━━━━━━━━━━━━━━━━━━━━ 59s 3s/step - categorical_accuracy: 0.2423 - loss: 18.1749 - val_categorical_accuracy: 0.2708 - val_loss: 1.5992 - learning_rate: 0.0010
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - categorical_accuracy: 0.4488 - loss: 1.3515
Epoch 2: val_loss improved from 1.59916 to 1.51866, saving model to conv3d_best_model.keras

Epoch 2: val_loss improved from 1.59916 to 1.51866, saving model to conv3d_best_weights.weights.h5
20/20 ━━━━━━━━━━━━━━━━━━━━ 57s 3s/step - categorical_accuracy: 0.4493 - loss: 1.3500 - val_categorical_accuracy: 0.2836 - val_loss: 1.5187 - learning_rate: 0.0010
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━

In [19]:
# Save the model weights manually after training
conv3d_model.save_weights('conv3d_final_weights.weights.h5')

In [20]:

# Load the best saved model
from keras.models import load_model

# Load best model and evaluate on validation set
#best_model = load_model('conv3d_best_model.h5
best_model = load_model('conv3d_best_model.keras')

# Evaluate the model
val_loss, val_acc = best_model.evaluate(val_generator, steps=validation_steps)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 616ms/step - categorical_accuracy: 0.6907 - loss: 1.0535
Validation Loss: 1.3327564001083374, Validation Accuracy: 0.5671641826629639



### Conclusion

In this notebook, we have built and trained two architectures for gesture recognition using video data: Conv3D and CNN + RNN (GRU). Both architectures were tested, and the best performing model was saved. Further experiments can be conducted to fine-tune the hyperparameters and improve accuracy.
